### Exploring OSM `surface` tags

In this notebook, I perform a custom OSM Overpass API query for all "driveable" roads that have surface labels. My main curiosity is which `surface` tags appear the most, and if I can identify an easy set of `surface` tag values that will account for the overwhelming majority of roads in the US.

First, I kick off a Docker container to host the OSM Overpass API. This query requires far too much data to use any Overpass API that is hosted online.

In [20]:
%%bash
# https://hub.docker.com/r/wiktorn/overpass-api
# http://localhost:12345/api/interpreter
docker run \
  -e OVERPASS_META=yes \
  -e OVERPASS_MODE=init \
  -e OVERPASS_PLANET_URL=file:///data/gis/us-latest.osm.bz2 \
  -e OVERPASS_RULES_LOAD=10 \
  -e OVERPASS_SPACE=55000000000 \
  -e OVERPASS_MAX_TIMEOUT=86400 \
  -v /data/gis:/data/gis \
  -v /data/gis/overpass_db:/db \
  -p 12345:80 \
  -d --rm --name overpass_usa wiktorn/overpass-api:latest

7b463d293ec20d6fab599eb3b2dd15f0463d89f6192df9ff25a9ec32d8317b6e


Let's define a class that will allow us to perform an OSM Overpass API query for drivable road networks:

In [14]:
from rsc.osm.overpass_api import OSMOverpassQuery, OSMOverpassResult


class OSMCustomOverpassQuery(OSMOverpassQuery):
    """ Custom OSM Overpass API query for (hopefully) drivable road networks """

    __slots__ = ['_highway_tags']

    DEFAULT_HIGHWAY_TAGS = [
        'motorway', 'motorway_link', 'motorway_junction', 'trunk',
        'trunk_link', 'primary', 'primary_link', 'secondary', 'secondary_link',
        'tertiary', 'tertiary_link', 'unclassified', 'residential'
    ]

    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self._highway_tags = kwargs.get('highway_tags',
                                        self.DEFAULT_HIGHWAY_TAGS)

    def perform_query(self) -> 'OSMOverpassResult':
        """ Perform an OSM Overpass API Request! """
        return OSMOverpassResult(self._perform_query())

    @property
    def _query_str(self) -> str:
        return f"""
            [out:{self._format}]
            [timeout:{self._timeout}]
            [maxsize:2147483648];
            (way["highway"]
            ["area"!~"yes"]
            ["access"!~"private"]
            ["highway"~"{'|'.join(self._highway_tags)}"]
            ["motor_vehicle"!~"no"]
            ["motorcar"!~"no"]
            ["surface"!~""]
            ["service"!~"alley|driveway|emergency_access|parking|parking_aisle|private"]
            (poly:'{self._poly_query_str}');
            >;
            );
            out;
        """

Now we can perform the query. This is a *very* broad query and therefore takes quite a bit of time.

In [17]:
import pathlib
out_path = pathlib.Path('/data/gis/us_road_surface/us_w_road_surface.osm')

if not out_path.is_file():
    # Setup custom query to local interpreter
    # Set a very long timeout
    q = OSMCustomOverpassQuery(format='xml', timeout=24 * 60 * 60)
    q.set_endpoint('http://localhost:12345/api/interpreter')

    # Use rough USA bounds for query
    with open('/data/gis/us_wkt.txt', 'r') as f:
        us_wkt = f.read()
    q.set_poly_from_wkt(us_wkt)

    # Perform query and save! This will take a long time.
    print('Performing query...')
    result = q.perform_query()
    print('Saving to file...')
    result.to_file(out_path)
else:
    print('OSM data found at %s' % str(out_path))

OSM data found at /data/gis/us_road_surface/us_w_road_surface.osm


At this point we can stop our Docker container hosting the Overpass API:

In [21]:
%%bash
docker stop overpass_usa

overpass_usa


And now we use ogr2ogr to convert the OSM file we downloaded to a CSV file for easier GIS processing (the OGR OSM driver is very limited). This can be done in Python too, but the command line tool is easier for simple file conversions.

In [31]:
%%bash
DATA_DIR=/data/gis/us_road_surface
OSM_PATH=$DATA_DIR/us_w_road_surface.osm
GPKG_PATH=$DATA_DIR/us_w_road_surface.gpkg
if [ ! -f $GPKG_PATH ]; then
    echo "Converting $OSM_PATH to $GPKG_PATH..."
    ogr2ogr $GPKG_PATH $OSM_PATH lines
else
    echo "File found: $GPKG_PATH"
fi

File found: /data/gis/us_road_surface/us_w_road_surface.gpkg


Let's parse the GPKG file to understand what surface types we are dealing with:

In [2]:
from osgeo import gdal, osr
from tqdm import tqdm

# Variable to store feature data that we load
feature_data = []

# WGS84 Spatial Reference: all of OSM is EPSG:4326
srs_wgs84 = osr.SpatialReference()
srs_wgs84.ImportFromEPSG(4326)

# Dataset is ogr OSM parsed file with "lines" layer exported
ds = gdal.OpenEx('/data/gis/us_road_surface/us_w_road_surface.gpkg')
layer = ds.GetLayer()
feature_count = layer.GetFeatureCount()
print('Loading & filtering dataset features...')
for idx in tqdm(range(feature_count)):
    # Get geometry, OSM ID, highway, and surface tag from each way
    feature = layer.GetNextFeature()
    highway = str(feature.GetField(2))
    wkt_str = feature.GetGeometryRef().ExportToWkt()
    osm_id = int(feature.GetField(0))
    other_tags = str(feature.GetField(8))

    # NOTE: parsing the misc. tags field is messy. This is about
    # as good as it gets.
    tags_dict = dict([[f.replace('"', '') for f in e.split('"=>"')]
                        for e in other_tags.split('","')])
    surface_type = tags_dict.get('surface', 'unknown')

    # Add to the feature data
    feature_data.append([osm_id, wkt_str, highway, surface_type])

# Close dataset
layer = None
ds = None

Loading & filtering dataset features...


100%|██████████| 2458128/2458128 [02:15<00:00, 18099.85it/s]


Empirically, if I take any `surface` labels that appear more than 1000 times, I get a reasonable set of labels that covers the fast majority of cases. Nice!

In [3]:
import pandas as pd

df = pd.DataFrame(feature_data, columns=['osm_id', 'wkt', 'highway', 'surface']).set_index('osm_id')
unique_surface = df['surface'].value_counts()
unique_surface = unique_surface[unique_surface > 1000]
unique_surface

asphalt            1639190
unpaved             266035
paved               230889
concrete            153371
gravel              105499
dirt                 28363
concrete:plates      14310
compacted             8585
paving_stones         2562
ground                2353
bricks                1751
Name: surface, dtype: int64

Let's see what percentage of `surface`-tagged roads this set of labels covers:

In [4]:
pc = unique_surface.sum() / len(feature_data)
print(f'Filtered surface tags account for {pc:.1%} of all driveable ways.')

Filtered surface tags account for 99.8% of all driveable ways.


I can live with that.

Let's save these filtered tags into a GPKG file for our classifier dataset prep.

In [ ]:
# Start a new dataset
print('Saving filtered features...')
driver = ogr.GetDriverByName('GPKG')
ds = driver.CreateDataSource(str(filtered_gpkg_path))
layer = ds.CreateLayer('roads', srs=srs_wgs84, geom_type=ogr.wkbLineString)

# Define fields
id_field = ogr.FieldDefn('osmid', ogr.OFTInteger64)
highway_field = ogr.FieldDefn('highway', ogr.OFTString)
surface_field = ogr.FieldDefn('surface', ogr.OFTString)
for field in (id_field, highway_field, surface_field):
    layer.CreateField(field)

# Add features
feature_defn = layer.GetLayerDefn()
for idx, (osm_id, wkt_str, highway,
            surface_type) in tqdm(enumerate(feature_data)):

    # New feature
    feat = ogr.Feature(feature_defn)

    # Set geometry
    geom = ogr.CreateGeometryFromWkt(wkt_str)
    feat.SetGeometry(geom)

    # Set fields
    feat.SetField('osmid', osm_id)
    feat.SetField('highway', highway)
    feat.SetField('surface', surface_type)

    # Flush
    layer.CreateFeature(feat)
    feat = None

# Close dataset
layer = None
ds = None